In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
from torch.utils.data import DataLoader
import torch
from torch.autograd import Variable
from faster_rcnn.utils.cython_bbox import bbox_overlaps
from pycrayon import CrayonClient

import cPickle
from torch.optim import SGD, RMSprop, Adam
from torch.optim.lr_scheduler import StepLR
from datetime import datetime
from faster_rcnn.utils.datasets.adapter import convert_data
from faster_rcnn.utils.evaluate.metter import AverageMeter
from faster_rcnn.utils.display.images import imshow, result_show

In [3]:
with open('sorted_index', 'rb') as fp:
    sorted_index = cPickle.load(fp)

### Đọc dữ liệu từ MS COCO dataset


In [4]:
from faster_rcnn.utils.datasets.adapter import convert_data

In [5]:
import os
import torchvision.transforms as transforms
from faster_rcnn.utils.datasets.mscoco.dataset import CocoData
from torch.utils.data import DataLoader



dataDir = './data/mscoco'
dataType = 'train2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
batch_size = 4

images_dir = os.path.join(dataDir,'images', dataType)
cap = CocoData(root = images_dir, annFile = annFile,sorted_indexes=sorted_index)

train_data_loader = DataLoader(cap, batch_size=batch_size, shuffle=False, collate_fn=convert_data)

loading annotations into memory...
Done (t=9.97s)
creating index...
index created!


In [ ]:
import time
s=time.time()
for i_batch, sample_batched in enumerate(train_data_loader):
    print(i_batch)
    if(i_batch > 200):
        break
    
e=time.time()
print(e-s)

0
1
2
3
4
5


No handlers could be found for logger "faster_rcnn.utils.datasets.mscoco.dataset"


6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152


In [6]:
print cap[28]

{'gt_classes': tensor([ 28,   1], dtype=torch.int32), 'boxes': tensor([[  267.2986,   307.1090,   318.4834,   494.7867],
        [   45.4976,    22.7488,   543.1280,  1401.8957]], dtype=torch.float64), 'tensor': tensor([[[[ 1.7352,  1.7352,  1.7352,  ...,  1.2728,  1.2728,  1.2728],
          [ 1.7352,  1.7352,  1.7352,  ...,  1.2728,  1.2728,  1.2728],
          [ 1.7352,  1.7352,  1.7352,  ...,  1.2728,  1.2728,  1.2728],
          ...,
          [ 0.6049,  0.6049,  0.6049,  ...,  0.6563,  0.6563,  0.6563],
          [ 0.6049,  0.6049,  0.6049,  ...,  0.6563,  0.6563,  0.6563],
          [ 0.6049,  0.6049,  0.6049,  ...,  0.6563,  0.6563,  0.6563]],

         [[ 1.9209,  1.9209,  1.9209,  ...,  1.5182,  1.5182,  1.5182],
          [ 1.9209,  1.9209,  1.9209,  ...,  1.5182,  1.5182,  1.5182],
          [ 1.9209,  1.9209,  1.9209,  ...,  1.5182,  1.5182,  1.5182],
          ...,
          [ 0.8354,  0.8354,  0.8354,  ...,  0.8880,  0.8880,  0.8880],
          [ 0.8354,  0.8354,  0.8354

In [7]:
dataDir = './data/mscoco'
dataType = 'val2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
batch_size = 6

images_dir = os.path.join(dataDir,'images', dataType)
val_cap = CocoData(root = images_dir, annFile = annFile)

val_data_loader = DataLoader(val_cap, batch_size=4, shuffle=False, collate_fn=default_collate)


loading annotations into memory...
Done (t=7.96s)
creating index...
index created!


Thử hiển thị ảnh cùng các bounding boxes

In [8]:
from faster_rcnn.faster_rcnn import FastRCNN


### Tính toán feed-forward


Chúng ta sử dụng một ảnh có kích thước đầu vào là  `(width , height) = (600, 800)`

Input:
    - im_data : 
        kích thước : (batch_size, dim, witdh, height)
    - ground_boxes: 
        kích thước (n, 4)
        

In [9]:
categories = ['__background__'] + [x['name'] for x in cap.coco.loadCats(cap.coco.getCatIds())]

In [10]:
net = FastRCNN(categories, debug=False)
net.cuda()
net.train()

/opt/conda/envs/pytorch/lib/python2.7/site-packages/torch/nn/parallel/data_parallel.py:24: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


FastRCNN(
  (rpn): RPN(
    (features): DataParallel(
      (module): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace)
        (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (15): 

In [11]:
param = filter(lambda x: x.requires_grad, net.parameters())
optimizer = SGD(param, lr=1e-3, momentum=0.9, weight_decay=0.0005)
exp_lr_scheduler = StepLR(optimizer, step_size=1000, gamma=0.95)

In [ ]:
def evaluate(data_gen ,model, steps_per_epoch=2000, epochs=1):
    model.eval()
    val_loss = AverageMeter()
    val_cross_entropy = AverageMeter()
    val_loss_box = AverageMeter()
    val_rpn_loss = AverageMeter()
    
    for epoch in range(epochs):
        for step in range(1, steps_per_epoch +1):
            blobs = data_gen.next()
            batch_tensor, im_info, batch_boxes, batch_boxes_index = convert_data(blobs)
            cls_prob, bbox_pred, rois = model(batch_tensor, im_info, batch_boxes, batch_boxes_index)

            loss = model.loss
            
            val_loss_box.update(model.loss_box.item())
            val_cross_entropy.update(model.cross_entropy.item())
            val_loss.update(loss.item())
            val_rpn_loss.update(model.rpn.loss.item())
    
    log_text = 'val_loss: %.4f' % (val_loss.avg)
    print(log_text)
    return val_loss ,val_cross_entropy, val_loss_box, val_rpn_loss


In [ ]:
def train(train_data_gen, val_data_gen, optimizer, lr_scheduler ,model, epochs, steps_per_epoch, val_step_per_epoch):
    
    exp_name = datetime.now().strftime('vgg16_%m-%d_%H-%M')
    cc = CrayonClient(hostname="crayon", port=8889)
    exp = cc.create_experiment(exp_name)
    
    
    train_loss = AverageMeter()
    cross_entropy = AverageMeter()
    loss_box = AverageMeter()
    rpn_loss = AverageMeter()
    current_step = 0
    
    for epoch in range(epochs):
        model.train()
        for step in range(1, steps_per_epoch +1):
            lr_scheduler.step()        
            blobs = data_gen.next()
            batch_tensor, im_info, batch_boxes, batch_boxes_index = convert_data(blobs)

            cls_prob, bbox_pred, rois = model(batch_tensor, im_info, batch_boxes, batch_boxes_index)
            cls_data = cls_prob.data.cpu().numpy()
            max_class_idx = np.argmax(cls_data, axis=1)
            loss = model.loss
            cross_entropy.update(model.cross_entropy.item())
            loss_box.update(model.loss_box.item())
            train_loss.update(loss.item())
            rpn_loss.update(model.rpn.loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            current_step = epoch * steps_per_epoch + step
            if step % 20 == 1:
                log_text = 'epoch: %d : step %d,  loss: %.4f at %s' % (
                    epoch + 1, step , train_loss.avg, datetime.now().strftime('%m\%d_%H:%M'))
                print(log_text)

            if step % 20 == 0:
                exp.add_scalar_value('train_loss', train_loss.avg, step=current_step)
                exp.add_scalar_value('rpn_loss', rpn_loss.avg, step=current_step)
                exp.add_scalar_value('cross_entropy', cross_entropy.avg, step=current_step)
                exp.add_scalar_value('loss_box', loss_box.avg, step=current_step)


                
        torch.save(model.state_dict(), './checkpoints/faster_model_at_epoch_%s.pkl' % (epoch + 1)) 
        val_loss ,val_cross_entropy, val_loss_box, val_rpn_loss = evaluate(val_data_gen, model, val_step_per_epoch)
        exp.add_scalar_value('val_loss', val_loss.avg, step=current_step)
        exp.add_scalar_value('val_rpn_loss', val_rpn_loss.avg, step=current_step)
        exp.add_scalar_value('val_cross_entropy', val_cross_entropy.avg, step=current_step)
        exp.add_scalar_value('val_loss_box', val_loss_box.avg, step=current_step)

In [ ]:
train(train_data_generator, val_data_generator ,optimizer=optimizer,lr_scheduler=exp_lr_scheduler, model=net, epochs=200, steps_per_epoch=6000, val_step_per_epoch=1000)

In [ ]:
net.load_state_dict(torch.load('faster.pkl'))

In [ ]:
pred_boxes, scores, classes, rois, im_data = net.detect("./test7.jpg", thr=0.5)
print classes
print scores
print pred_boxes
imshow(im_data[0],[] ,pred_boxes, random=True)

In [ ]:
print 1

In [ ]:
__